In [ ]:
import pandas as pd
import requests
import logging
import time
import os

# Configuração do logging
logging.basicConfig(
    filename='consulta_cnpj.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Arquivo de saída CSV
output_file = 'consultas_cnpj_resultados.csv'

# Verificar se o arquivo de saída já existe e carregar CNPJs já consultados
if os.path.exists(output_file):
    df_existente = pd.read_csv(output_file, dtype=str)  # Garantir que os CNPJs sejam tratados como strings
    cnpjs_consultados = set(df_existente['CNPJ'].tolist())
else:
    cnpjs_consultados = set()

# Função para consultar o CNPJ
def consultar_cnpj(cnpj):
    url = f'https://api.cnpja.com/office/{cnpj}?registrations=BR'
    headers = {
        'Authorization': '5e51dcc2-c8d0-4e2c-b158-b34c057c1ba9-a04c3c9c-3ed5-4359-971e-67f7bafb5b3c'
    }
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            logging.info(f"Consulta realizada com sucesso para o CNPJ: {cnpj}")
            return response.json()
        else:
            logging.error(f"Erro ao consultar CNPJ {cnpj}: {response.status_code}")
            return None
    except Exception as e:
        logging.error(f"Erro ao tentar consultar o CNPJ {cnpj}: {e}")
        return None

# Função para extrair os dados para o formato de dicionário
def extrair_dados_para_df(dados_cnpj):
    dados = {
        'CNPJ': dados_cnpj['taxId'],
        'Nome': dados_cnpj['company']['name'],
        'Nome Fantasia': dados_cnpj.get('alias', 'Não disponível'),
        'Capital Social': dados_cnpj['company']['equity'],
        'Natureza Jurídica': dados_cnpj['company']['nature']['text'],
        'Tamanho': dados_cnpj['company']['size']['text'],
        'Data de Fundação': dados_cnpj['founded'],
        'Status': dados_cnpj['status']['text'],
        'Data de Status': dados_cnpj['statusDate'],
        'Razão de Status': dados_cnpj.get('reason', {}).get('text', 'Não disponível'),
        'Endereço': f"{dados_cnpj['address']['street']}, {dados_cnpj['address']['number']} - {dados_cnpj['address']['district']}, {dados_cnpj['address']['city']}/{dados_cnpj['address']['state']}, {dados_cnpj['address']['zip']}",
        'País': dados_cnpj['address']['country']['name'],
        'Telefone': ', '.join([f"({telefone['area']}) {telefone['number']}" for telefone in dados_cnpj['phones']]),
        'Email': ', '.join([email['address'] for email in dados_cnpj['emails']]),
        'Atividade Principal': dados_cnpj['mainActivity']['text'],
        'Atividades Secundárias': ', '.join([activity['text'] for activity in dados_cnpj['sideActivities']]) if dados_cnpj['sideActivities'] else 'Nenhuma',
    }

    # Inscrição Estadual
    if 'registrations' in dados_cnpj and len(dados_cnpj['registrations']) > 0:
        inscricao_estadual = dados_cnpj['registrations'][0]
        dados['Inscrição Estadual Estado'] = inscricao_estadual['state']
        dados['Inscrição Estadual Número'] = inscricao_estadual['number']
        dados['Inscrição Estadual Status'] = inscricao_estadual['status']['text']
        dados['Inscrição Estadual Tipo'] = inscricao_estadual['type']['text']
        dados['Inscrição Estadual Data de Status'] = inscricao_estadual['statusDate']
    else:
        dados['Inscrição Estadual Estado'] = 'Não encontrada'
        dados['Inscrição Estadual Número'] = 'Não encontrada'
        dados['Inscrição Estadual Status'] = 'Não encontrada'
        dados['Inscrição Estadual Tipo'] = 'Não encontrada'
        dados['Inscrição Estadual Data de Status'] = 'Não encontrada'

    return dados

# Função para limpar os CNPJs
def limpar_cnpj(cnpj):
    return ''.join(e for e in str(cnpj) if e.isdigit())  # Remove qualquer caractere não numérico

# Carregar o arquivo Excel
file_path = r"C:\Users\leonardo.campos\Downloads\Hunting List_RIGARRRJ.xlsx"
df_excel = pd.read_excel(file_path)

# Contador para controle de tempo
contador = 0

# Iterar sobre a coluna 'CNPJ' e consultar as informações
for cnpj in df_excel['CNPJ']:  # Supondo que a coluna CNPJ tenha esse nome
    cnpj_limpo = limpar_cnpj(cnpj)  # Limpa o CNPJ para remover ./-

    # Se o CNPJ já foi consultado, pular
    if cnpj_limpo in cnpjs_consultados:
        logging.info(f"CNPJ {cnpj_limpo} já consultado. Pulando...")
        continue

    logging.info(f"Iniciando consulta para o CNPJ: {cnpj_limpo}")

    dados_cnpj = consultar_cnpj(cnpj_limpo)
    if dados_cnpj:
        dados_empresa = extrair_dados_para_df(dados_cnpj)

        # Criar DataFrame e registrar no CSV imediatamente
        df_resultado = pd.DataFrame([dados_empresa])

        if not os.path.exists(output_file):
            df_resultado.to_csv(output_file, index=False, mode='w', header=True)
        else:
            df_resultado.to_csv(output_file, index=False, mode='a', header=False)

        logging.info(f"Dados do CNPJ {cnpj_limpo} registrados no arquivo CSV.")

        # Adicionar CNPJ consultado ao conjunto para evitar duplicação
        cnpjs_consultados.add(cnpj_limpo)

    contador += 1

    # A cada 5 consultas, aguardar 10 segundos para continuar
    if contador % 5 == 0:
        logging.info("Aguardando 10 segundos antes de continuar...")
        time.sleep(5)

logging.info("Processo de consulta finalizado.")
